In [4]:
import sys
import torch
from torch import nn
from torchinfo import summary
from torchvision import transforms, datasets
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, random_split

import os
from pathlib import Path
from tqdm.auto import tqdm
from timeit import default_timer as timer

In [5]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count() or 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# Resizing to the 224x224 standard and using normalization 

simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform_trivial_augment = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

In [7]:
venv_dir = Path(sys.prefix)
project_root = venv_dir.parent
image_path = project_root/"data/food-101-pizzasteaksushi/images"

dataset = datasets.ImageFolder(
    root=image_path,
    transform=train_transform_trivial_augment,
)

train_size = int(0.8*len(dataset))
train_dataset, test_dataset = random_split(dataset=dataset, lengths=[train_size, len(dataset)-train_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [8]:
def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer):
    model.train()
    train_loss, train_acc = 0, 0

    for (X, y) in iter(dataloader):
        X, y = X.to(DEVICE), y.to(DEVICE)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_acc /= len(dataloader)
    train_loss /= len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module):
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for (X, y) in iter(dataloader):
            X, y = X.to(DEVICE), y.to(DEVICE)

            y_pred = model(X)

            loss = loss_fn(y_pred, y)
            test_loss += loss.item()

            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            test_acc += (y_pred_class == y).sum().item()/len(y_pred)

    test_acc /= len(dataloader)
    test_loss /= len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, 
          loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, scheduler=None,
          epochs=5):
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn)
        if scheduler is not None:
            scheduler.step()

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

In [10]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model = torch.compile(model)

# resnet18 is pretrained on 1000 classes, we will modify is final fully connected layer for 3 classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)

model = model.to(DEVICE)
print(summary(model))

# Freezing base layers parameters, to only train the final layer
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

NUM_EPOCHS = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)

start_time = timer()
model_results = train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs=NUM_EPOCHS)
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

Layer (type:depth-idx)                        Param #
OptimizedModule                               --
├─ResNet: 1-1                                 --
│    └─Conv2d: 2-1                            9,408
│    └─BatchNorm2d: 2-2                       128
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
│    └─Sequential: 2-5                        --
│    │    └─BasicBlock: 3-1                   73,984
│    │    └─BasicBlock: 3-2                   73,984
│    └─Sequential: 2-6                        --
│    │    └─BasicBlock: 3-3                   230,144
│    │    └─BasicBlock: 3-4                   295,424
│    └─Sequential: 2-7                        --
│    │    └─BasicBlock: 3-5                   919,040
│    │    └─BasicBlock: 3-6                   1,180,672
│    └─Sequential: 2-8                        --
│    │    └─BasicBlock: 3-7                   3,673,088
│    │    └─BasicBlock: 3-8                   4,720,640
│    └─AdaptiveA

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0392 | train_acc: 0.4658 | test_loss: 0.9473 | test_acc: 0.5811
Epoch: 2 | train_loss: 0.8841 | train_acc: 0.6496 | test_loss: 0.8344 | test_acc: 0.6743
Epoch: 3 | train_loss: 0.7943 | train_acc: 0.7217 | test_loss: 0.7331 | test_acc: 0.7593
Epoch: 4 | train_loss: 0.7112 | train_acc: 0.7692 | test_loss: 0.6775 | test_acc: 0.7933
Epoch: 5 | train_loss: 0.6603 | train_acc: 0.7854 | test_loss: 0.6329 | test_acc: 0.7993
Epoch: 6 | train_loss: 0.6113 | train_acc: 0.8129 | test_loss: 0.5944 | test_acc: 0.8158
Epoch: 7 | train_loss: 0.5867 | train_acc: 0.8054 | test_loss: 0.5648 | test_acc: 0.8279
Epoch: 8 | train_loss: 0.5620 | train_acc: 0.8179 | test_loss: 0.5523 | test_acc: 0.8262
Epoch: 9 | train_loss: 0.5263 | train_acc: 0.8317 | test_loss: 0.5265 | test_acc: 0.8300
Epoch: 10 | train_loss: 0.5173 | train_acc: 0.8333 | test_loss: 0.5006 | test_acc: 0.8503
Epoch: 11 | train_loss: 0.5058 | train_acc: 0.8267 | test_loss: 0.4822 | test_acc: 0.8531
Epoch: 12 | train_l

Model achieves 85% accuracy, I will now do some fine-tuning

In [ ]:
# Unfreezing the last ResNet block, lowering learning rate and giving the optimizer only the unfrozen parameters with separate learning rates

for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

optimizer = torch.optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 0.0001},
    {'params': model.fc.parameters(), 'lr': 0.001}
], momentum=0.9)


# Improving data augmentation
better_train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

venv_dir = Path(sys.prefix)
project_root = venv_dir.parent
image_path = project_root/"data/food-101-pizzasteaksushi/images"

dataset = datasets.ImageFolder(
    root=image_path,
    transform=better_train_transform,
)

train_size = int(0.8*len(dataset))
train_dataset, test_dataset = random_split(dataset=dataset, lengths=[train_size, len(dataset)-train_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)


# Adding learning rate scheduling and using it in training loop
# Gives smoother scheduling compared to StepLR
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=NUM_EPOCHS)

NUM_EPOCHS = 20

start_time = timer()
model_results = train(model, train_dataloader, test_dataloader, loss_fn, optimizer, scheduler=scheduler, epochs=NUM_EPOCHS)
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

NameError: name 'model' is not defined

In [ ]:
venv_dir = Path(sys.prefix)
project_root = venv_dir.parent
models_path = project_root/"trained_models"

torch.save(model.state_dict(), models_path/"resnet18_pizzasushisteak.pth")